# QUIZ - 2024-08

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# import modules
from service.models import *
from django.db.models import *
from django.db.models.functions import *
from datetime import datetime, date, time
import json

## PART 1 - CRUD Operations
1.1 สร้างนัดหมาย (`Appointment`) ของลูกค้า "Customer 1" ซึ่งต้องการจองบริการ "Haircut" ของ "Provider 2"

โดยสร้างนัดหมายในวันที่ 2024-10-01 เวลา 11:00 น. 

(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [19]:
w_service = Service.objects.get(
    service_provider__name = "Provider 2",
    name = "Haircut"
)

w_customer = Customer.objects.get(
    name = "Customer 1"
)

appointment = Appointment.objects.create(
    customer = w_customer,
    service = w_service,
    appointment_date = date(2024, 10, 1),
    appointment_time = time(11)
)

In [20]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-01, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.2 ทำการแก้ไขนัดหมายที่เพิ่งสร้างใน 1.1 โดยเปลี่ยนวันที่จากวันที่ 2024-10-01 เป็น 2024-10-10 เวลาเดิม

(0.25 คะแนน)

In [21]:
appointment.appointment_date = date(2024, 10, 10)
appointment.save()

In [22]:
# Check result
res = Appointment.objects.last()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-10-10, Appoint Time: 11:00:00
Service Name: Haircut, Provider Name: Provider 2, Customer Name: Customer 1


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-14 ของ "Customer 4" โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Manicure" ของ "Provider 3" เป็น "Massage" ของ "Provider 4" แทน

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [25]:
appointment = Appointment.objects.get(
    customer__name = "Customer 4",
    appointment_date = date(2024, 8, 14)
)

w_service = Service.objects.get(
    name = "Massage",
    service_provider__name = "Provider 4"
)

appointment.service = w_service
appointment.save()

In [26]:
# Check result
res = Appointment.objects.get(customer__name="Customer 4", appointment_date=date(2024,8,14))
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-14, Appoint Time: 10:00:00
Service Name: Massage, Provider Name: Provider 4, Customer Name: Customer 4


1.4 ทำตามขั้นตอนดังนี้

1. สร้างหมวดหมู่บริการ (`ServiceCategory`) ใหม่ชื่อ "Facial & Body Care" และมีรายละเอียดว่า "Services related to face & body care" 
2. สร้าง บริการ (`Service`) ใหม่ดังนี้ 

    - Name: "Face Massage"
    - Description: "Special face massage!"
    - Price: 150.00
    - โดยบริการนี้ให้บริการโดย "Provider 1" และ "Provider 2" (Hint: ดังนั้นต้องสร้าง 2 รายการ)

3. เพิ่มบริการใหม่ทั้ง 2 รายการเข้าไปในหมวดหมู่ "Facial & Body Care"
4. ทำการย้ายบริการ (`Service`) ในหมวดหมู่บริการ (`ServiceCategory`) "Body Care" ไปที่ หมวดหมู่ "Facial & Body Care" ที่สร้างขึ้นมาใหม่
5. ทำการลบหมวดหมู่บริการ "Body Care"

(1 คะแนน)

In [30]:
# 01 create new ServiceCategory
facial = ServiceCategory.objects.create(
    name = "Facial & Body Care",
    description = "Services related to face & body care"
)

In [31]:
# 02 create new Service
prov1 = ServiceProvider.objects.get(name = "Provider 1")
prov2 = ServiceProvider.objects.get(name = "Provider 2")

mass_prov1 = Service.objects.create(
    service_provider = prov1,
    name = "Face Massage",
    description = "Special face massage!",
    price = 150
)

mass_prov2 = Service.objects.create(
    service_provider = prov2,
    name = "Face Massage",
    description = "Special face massage!",
    price = 150
)

In [33]:
# 03 add category
facial.services.add(mass_prov1)
facial.services.add(mass_prov2)

In [48]:
# 04 move "Body Care" to "Facial & Body Care"
body_care = ServiceCategory.objects.get(name = "Body Care")
body_services = body_care.services.all()

for s in body_services:
    body_care.services.remove(s)
    facial.services.add(s)

In [49]:
# 05 delete "Body Care"
body_care.delete()

(1, {'service.ServiceCategory': 1})

In [50]:
# Check result
sc1 = ServiceCategory.objects.get(name="Facial & Body Care")
sc1.services.count()

7

## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02 

(0.5 คะแนน)

In [8]:
appointment = Appointment.objects.filter(
    created_at__range = (date(2024, 8, 1), date(2024, 8, 2))
).aggregate(count = Count('id'))

appointment

{'count': 4}

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายท่ีมีการนัดหมาย (`Appointment.appoint_date`) ในช่วงวันที่ 2024-08-10 ถึง 2024-08-15

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Massage, Provider: Provider 2
Name: Massage, Provider: Provider 3
Name: Massage, Provider: Provider 4
```

In [15]:
appointments = Appointment.objects.filter(
    appointment_date__range = (date(2024, 8, 10), date(2024, 8, 15))
).order_by("service__service_provider")

for a in appointments:
    print(f"Name: {a.service.name}, Provider: {a.service.service_provider.name}")

Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 2
Name: Massage, Provider: Provider 2
Name: Manicure, Provider: Provider 3
Name: Massage, Provider: Provider 3


2.3 ให้แสดงผลข้อมูลบริการ (`Service`) โดยเรียงลำดับตามราคา (`Service.price`) จากน้อยไปมาก ดังในตัวอย่าง

```python
[
    {
        "price": 100,
        "full_name": "Massage (Provider 1)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 2)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 3)"
    },
    {
        "price": 100,
        "full_name": "Massage (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 4)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 1)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 3)"
    },
    {
        "price": 30,
        "full_name": "Manicure (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 4)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 2)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 3)"
    },
    {
        "price": 20,
        "full_name": "Haircut (Provider 1)"
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [22]:
services = Service.objects.annotate(
    full_name = Concat(F('name'), Value(' ('), F('service_provider__name'), Value(')')),
).order_by("-price")

print(json.dumps(list(services.values()), indent=4, sort_keys=False))

[
    {
        "id": 12,
        "service_provider_id": 4,
        "name": "Massage",
        "description": "Full body massage",
        "price": 100,
        "full_name": "Massage (Provider 4)"
    },
    {
        "id": 6,
        "service_provider_id": 2,
        "name": "Massage",
        "description": "Full body massage",
        "price": 100,
        "full_name": "Massage (Provider 2)"
    },
    {
        "id": 3,
        "service_provider_id": 1,
        "name": "Massage",
        "description": "Full body massage",
        "price": 100,
        "full_name": "Massage (Provider 1)"
    },
    {
        "id": 9,
        "service_provider_id": 3,
        "name": "Massage",
        "description": "Full body massage",
        "price": 100,
        "full_name": "Massage (Provider 3)"
    },
    {
        "id": 8,
        "service_provider_id": 3,
        "name": "Manicure",
        "description": "Nail trimming and polish",
        "price": 30,
        "full_name": "Manicure (Prov

2.4 ให้หาจำนวนนัดหมายของลูกค้าแต่ละคน

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()`*

**Expected Output**

```python
Name: Customer 4, Total Appoinment: 3
Name: Customer 2, Total Appoinment: 4
Name: Customer 3, Total Appoinment: 3
Name: Customer 1, Total Appoinment: 5
```

In [25]:
cus_appointment = Customer.objects.annotate(
    total_app = Count('appointment')
)

for c in cus_appointment:
    print(f"Name: {c.name}, Total Appoinment: {c.total_app}")

Name: Customer 4, Total Appoinment: 3
Name: Customer 2, Total Appoinment: 4
Name: Customer 3, Total Appoinment: 3
Name: Customer 1, Total Appoinment: 5


2.5 ให้หาว่าผู้ให้บริการ (`ServiceProvider`) แต่ละคนนั้นได้เงินโดยรวมเท่าไหร่

Hint: ดูจากนั้ดหมายว่ามีการนัดบริการ (`Appointment.service`) ของแต่ละผู้ให้บริการรวมเป็นยอดเงินเท่าไหร่

*หมายเหตุ: ไม่จำเป็นต้องใช้ `annotate()` สามารถ loop ได้เต็มที่เลยครับ*

**Expected Output**

```python
Name: Provider 4, Total Price: 20
Name: Provider 2, Total Price: 210
Name: Provider 3, Total Price: 130
Name: Provider 1, Total Price: 410
```

In [17]:
def showTotalPrice(all_appo):
    total = 0
    provider = ""
    for i in all_appo:
        total += i.service.price
        provider = i.service.service_provider.name
    print(f"Name: {provider}, Total Price: {total}")

In [18]:
total_prov1 = Appointment.objects.filter(service__service_provider__name = "Provider 1")
total_prov2 = Appointment.objects.filter(service__service_provider__name = "Provider 2")
total_prov3 = Appointment.objects.filter(service__service_provider__name = "Provider 3")
total_prov4 = Appointment.objects.filter(service__service_provider__name = "Provider 4")

showTotalPrice(total_prov1)
showTotalPrice(total_prov2)
showTotalPrice(total_prov3)
showTotalPrice(total_prov4)

Name: Provider 1, Total Price: 410
Name: Provider 2, Total Price: 210
Name: Provider 3, Total Price: 130
Name: Provider 4, Total Price: 20


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อหมวดหมู่บริการ (`ServiceCategory.name`) เป็นตัวใหญ่ (uppercase) ทั้งหมด

(0.5 คะแนน)

**Expected Output**

```python
Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
```

In [32]:
categories = ServiceCategory.objects.annotate(
    upper_name = Upper(F('name'))
)

for c in categories:
    print(f"Name: {c.name}, Upper Name: {c.upper_name}")

Name: Hair Care, Upper Name: HAIR CARE
Name: Nail Care, Upper Name: NAIL CARE
Name: Body Care, Upper Name: BODY CARE
